***GENERATED CODE FOR clustering PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Corona_Patient[sex]', 'transformation_label': 'String Indexer'}], 'feature': 'Corona_Patient[sex]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '80', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Corona_Patient[sex]'}, {'feature_label': 'Corona_Patient[sex]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Corona_Patient[sex]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Corona_Patient[age]', 'transformation_label': 'String Indexer'}], 'feature': 'Corona_Patient[age]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': '0s', 'max': '80s', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Corona_Patient[age]'}, {'feature_label': 'Corona_Patient[age]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Corona_Patient[age]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Corona_Patient[country]', 'transformation_label': 'String Indexer'}], 'feature': 'Corona_Patient[country]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Korea', 'max': 'Korea', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Corona_Patient[country]'}, {'feature_label': 'Corona_Patient[country]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Corona_Patient[country]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Corona_Patient[province]', 'transformation_label': 'String Indexer'}], 'feature': 'Corona_Patient[province]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Seoul', 'max': 'Seoul', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Corona_Patient[province]'}, {'feature_label': 'Corona_Patient[province]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Corona_Patient[province]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Corona_Patient[city]', 'transformation_label': 'String Indexer'}], 'feature': 'Corona_Patient[city]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Dongdaemun-gu', 'max': 'Yongsan-gu', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Corona_Patient[city]'}, {'feature_label': 'Corona_Patient[city]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Corona_Patient[city]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Corona_Patient[infection_case]', 'transformation_label': 'String Indexer'}], 'feature': 'Corona_Patient[infection_case]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'Dongan Church', 'max': 'overseas inflow', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Corona_Patient[infection_...'}, {'feature_label': 'Corona_Patient[infection_case]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Corona_Patient[infection_case]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Corona_Patient[state]', 'transformation_label': 'String Indexer'}], 'feature': 'Corona_Patient[state]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '80', 'mean': '', 'stddev': '', 'min': 'released', 'max': 'released', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Corona_Patient[state]'}, {'feature_label': 'Corona_Patient[state]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Corona_Patient[state]')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        stageAttributes['TrainingPercent']


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run clusteringHooks.ipynb
try:
	#sourcePreExecutionHook()

	clusteringhdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1698989521.csv', 'filename': 'tempFile1698989521.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(clusteringhdfs)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run clusteringHooks.ipynb
try:
	#transformationPreExecutionHook()

	clusteringautofe = TransformationMain.run(clusteringhdfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Corona_Patient[sex]", "transformation_label": "String Indexer"}], "feature": "Corona_Patient[sex]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Corona_Patient[sex]"}, {"transformationsData": [{"feature_label": "Corona_Patient[age]", "transformation_label": "String Indexer"}], "feature": "Corona_Patient[age]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "0s", "max": "80s", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Corona_Patient[age]"}, {"transformationsData": [{"feature_label": "Corona_Patient[country]", "transformation_label": "String Indexer"}], "feature": "Corona_Patient[country]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Korea", "max": "Korea", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Corona_Patient[country]"}, {"transformationsData": [{"feature_label": "Corona_Patient[province]", "transformation_label": "String Indexer"}], "feature": "Corona_Patient[province]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Seoul", "max": "Seoul", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Corona_Patient[province]"}, {"transformationsData": [{"feature_label": "Corona_Patient[city]", "transformation_label": "String Indexer"}], "feature": "Corona_Patient[city]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Dongdaemun-gu", "max": "Yongsan-gu", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Corona_Patient[city]"}, {"transformationsData": [{"feature_label": "Corona_Patient[infection_case]", "transformation_label": "String Indexer"}], "feature": "Corona_Patient[infection_case]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "Dongan Church", "max": "overseas inflow", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Corona_Patient[infection_..."}, {"transformationsData": [{"feature_label": "Corona_Patient[state]", "transformation_label": "String Indexer"}], "feature": "Corona_Patient[state]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "80", "mean": "", "stddev": "", "min": "released", "max": "released", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Corona_Patient[state]"}]}))

	#transformationPostExecutionHook(clusteringautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run clusteringHooks.ipynb
try:
	#mlPreExecutionHook()

	clusteringclustering = Clustering.run(clusteringautofe,spark,json.dumps( {"autoClustering": 0, "defaultclusters": 0, "TrainingPercent": "100", "model": "Kmeans", "originalfile": "/FileStore/platform/extra/tempFile1698989521.csv", "paascall": 1, "totalClusters": 2}))

	#mlPostExecutionHook(clusteringclustering)

except Exception as ex: 
	logging.error(ex)
#spark.stop()
